# Class notes for Nov 13, 2023

Reflecting the activities in class using the Omeka S API 

In [1]:
import json
import requests

This is the helper function to query `secrets.json`:

In [2]:
def get_credential(json_file, key, sub_key):
   try:
       with open(json_file) as f:
           data = json.load(f)
           return data[key][sub_key]
   except Exception as e:
       print("Error: ", e)

In [3]:
baseURL = 'http://jajohnst.projectst.si.umich.edu/omekas/api/' # modify this for your own site
key_identity = get_credential('../secrets.json', 'omeka', 'key_identity')
key_credential = get_credential('../secrets.json', 'omeka', 'key_credential')

Create a dictionary for those credentials:

In [4]:
omeka_credentials = {
    'key_identity': key_identity,
    'key_credential': key_credential
}

## Make a request to get item information

For a specific item, with the endpoint for `items` and ID `587`:

In [5]:
resource_type = 'items'
item_ID = 587

In [6]:
item_request = requests.get(baseURL + resource_type + '/' + str(item_ID), params=omeka_credentials)

In [7]:
item_request.status_code

200

### Looking into the response data

In [8]:
item_data = item_request.json()

In [9]:
item_data['o:id']

587

In [10]:
item_data['o:title']

'Main public library building, Detroit, Michigan. Cass Gilbert, architect. Postcard by Albertype Co., 1921-1930. Prints & Photographs Division'

In [11]:
for key, val in item_data.items():
    print(key,': ',val)

@context :  http://jajohnst.projectst.si.umich.edu/omekas/api-context
@id :  http://jajohnst.projectst.si.umich.edu/omekas/api/items/587
@type :  ['o:Item', 'dctype:Image']
o:id :  587
o:is_public :  True
o:owner :  {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/users/2', 'o:id': 2}
o:resource_class :  {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/resource_classes/26', 'o:id': 26}
o:resource_template :  None
o:thumbnail :  None
o:title :  Main public library building, Detroit, Michigan. Cass Gilbert, architect. Postcard by Albertype Co., 1921-1930. Prints & Photographs Division
thumbnail_display_urls :  {'large': 'http://jajohnst.projectst.si.umich.edu/omekas/files/large/52d9101c9e01f8841edea69d21d2f802c009d16c.jpg', 'medium': 'http://jajohnst.projectst.si.umich.edu/omekas/files/medium/52d9101c9e01f8841edea69d21d2f802c009d16c.jpg', 'square': 'http://jajohnst.projectst.si.umich.edu/omekas/files/square/52d9101c9e01f8841edea69d21d2f802c009d16c.jpg'}
o:created :  {

In [12]:
dctitle = item_data.get('dcterms:title')

In [13]:
dctitle

[{'type': 'literal',
  'property_id': 1,
  'property_label': 'Title',
  'is_public': True,
  '@value': 'Main public library building, Detroit, Michigan. Cass Gilbert, architect. Postcard by Albertype Co., 1921-1930. Prints & Photographs Division'}]

In [14]:
dctitle[0]['@value']

'Main public library building, Detroit, Michigan. Cass Gilbert, architect. Postcard by Albertype Co., 1921-1930. Prints & Photographs Division'

In [15]:
item_data['dcterms:title'][0]['@value']

'Main public library building, Detroit, Michigan. Cass Gilbert, architect. Postcard by Albertype Co., 1921-1930. Prints & Photographs Division'

## Making a change to information in the Omeka site

Using the API, you can also make requests that add, change, update, or delete information in the Omeka site's database.

Here, for example, is the case of adding a new `item_set`:

In [16]:
# get new keys

key_credential = get_credential('../secrets.json', 'omeka-si676-2023', 'key_credential')
key_identity = get_credential('../secrets.json', 'omeka-si676-2023', 'key_identity')

In [17]:
item_set_create = requests.post(baseURL + 'item_sets/', params={'key_credential': key_credential, 'key_identity': key_identity}, 
data='''{
    "dcterms:title" : [
        {
            "type" : "literal",
            "property_label" : "Title",
            "@value" : "NewItemSetfromAPI Python test",
            "property_id" : 1
        }
    ]
}''')

In [18]:
item_set_create.url

'http://jajohnst.projectst.si.umich.edu/omekas/api/item_sets?key_credential=hv8DgOtPiD9LrOFumjvoSfxoo9idXTdU&key_identity=rnHJtGWeGV3BmWnRM2WUyZBdBBKMLBWK'

In [19]:
item_set_create.status_code

415

In [20]:
item_set_create.headers

{'Date': 'Tue, 14 Nov 2023 14:21:59 GMT', 'Server': 'Apache', 'Omeka-S-Version': '3.2.3', 'Keep-Alive': 'timeout=5, max=99', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8'}

In [21]:
item_set_create.json()

{'errors': {'error': 'Invalid Content-Type header. Expecting "application/json", got "none".'}}

In [22]:
# add specific content type header
headers = {'Content-Type':'application/json'}

# rerun post request with headers
item_set_create = requests.post(baseURL + 'item_sets/', headers=headers, params={'key_credential': key_credential, 'key_identity': key_identity}, 
data='''{
    "dcterms:title" : [
        {
            "type" : "literal",
            "property_label" : "Title",
            "@value" : "NewItemSetfromAPI Python test",
            "property_id" : 1
        }
    ]
}''')

In [23]:
item_set_create.status_code

200

In [24]:
item_set_create.json()

{'@context': 'http://jajohnst.projectst.si.umich.edu/omekas/api-context',
 '@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/item_sets/675',
 '@type': 'o:ItemSet',
 'o:id': 675,
 'o:is_public': True,
 'o:owner': {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/users/1',
  'o:id': 1},
 'o:resource_class': None,
 'o:resource_template': None,
 'o:thumbnail': None,
 'o:title': 'NewItemSetfromAPI Python test',
 'thumbnail_display_urls': {'large': None, 'medium': None, 'square': None},
 'o:created': {'@value': '2023-11-14T14:23:32+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:modified': {'@value': '2023-11-14T14:23:32+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:is_open': False,
 'o:items': {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/items?item_set_id=675'},
 'dcterms:title': [{'type': 'literal',
   'property_id': 1,
   'property_label': 'Title',
   'is_public': True,
   '@value': 'NewItemSetfromAPI Python test'}

### Create multiple from a list

The point of using the API is to automate batch operations. So,
for example, if you want to create multiple resources, it can do that very quickly. 
Adapting the above example: 

In [25]:
# create list of things to create
new_item_set_names = ['Another Item Set from the REST API', 'Spam, Spam, Spam', 'A Set to Remember', 'That\'s all folks']

In [28]:
# add specific content type header
headers = {'Content-Type':'application/json'}
resource_type = 'item_sets'

# run the post request in a loop, with headers and a data in formatted string
for set_name in new_item_set_names:
    data = '{"dcterms:title" : [{"type" : "literal","property_label" : "Title","@value" : "' + set_name + '","property_id" : 1}]}'
    r = requests.post(baseURL + resource_type + '/', headers=headers, params={'key_credential': key_credential, 'key_identity': key_identity}, data=data)
    print('Requested new item set creation:', set_name)

Requested new item set creation: Another Item Set from the REST API
Requested new item set creation: Spam, Spam, Spam
Requested new item set creation: A Set to Remember
Requested new item set creation: That's all folks


# Success!

![Screenshot of the sets created, as viewed in the Omeka S Admin View](../assets/omeka-api-sets-created.jpg "Screenshot of the sets created in Omeka S Admin view")